### SimpleITK-based pipeline for BRATS image segmentation


In [ ]:
import os
import numpy as np
import SimpleITK as sitk
# for interactive visualization
import matplotlib.pyplot as plt
%matplotlib inline
from slice_viewer import ImageSliceViewer3D


DATADIR = "/home/shared/data/ITK-train"
RESULTDIR = "Results/ITK_train"

sequences = ("flair","t1","t2","t1ce", "label")

subject_id = "BraTS20_Training_030"  # replace with other subject_id
# these are the 20 Subjects from the training set:
#BraTS20_Training_010  BraTS20_Training_029  BraTS20_Training_075  BraTS20_Training_150  BraTS20_Training_180  BraTS20_Training_336  BraTS20_Training_365
#BraTS20_Training_021  BraTS20_Training_030  BraTS20_Training_108  BraTS20_Training_168  BraTS20_Training_233  BraTS20_Training_341  BraTS20_Training_366
#BraTS20_Training_027  BraTS20_Training_068  BraTS20_Training_124  BraTS20_Training_178  BraTS20_Training_253  BraTS20_Training_352

In [ ]:
# LOAD all images of this subject (including ground truth for later evaluation)
images = {}
for seq in sequences:
    filename = os.path.join(DATADIR, subject_id, f"{subject_id}_{seq}.nii.gz")
    print(f"Load image from {filename} ")  #  load and convert to float for images and uint8 for label image
    images[seq] = sitk.ReadImage(filename, outputPixelType=sitk.sitkFloat32 if seq != "label" else sitk.sitkUInt8)

### How to get help on specific SimpleITK functions or classes
`dir(sitk)` shows all attributes and functions of sitk.

`help()` displays the interface and a description for functions: For example: `help(sitk.RescaleIntensity)`



In [4]:
def evaluate_labels(prediction, ground_truth):
    """Compute measures for evaluation of segmentation masks.
    We assume the followin labels: 0 - background, 1-tumor, 2-edema.
    Parameters
    ----------
    prediction : sitk.Image
        predicted label image.
    ground_truth : sitk.Image | None
        ground_truth label image.
    Returns
    -------
    out_dirct: Dict
        a dictionary containing overlap measures for tumor, edema and tumor+edema, 
        see https://doi.org/10.54294/1vixgg for a description of the measures.
    """
    tumor_pred = sitk.BinaryThreshold(prediction, lowerThreshold=1, upperThreshold=1, insideValue=1, outsideValue=0)
    tumor_gt = sitk.BinaryThreshold(ground_truth, lowerThreshold=1, upperThreshold=1, insideValue=1, outsideValue=0)
    edema_pred = sitk.BinaryThreshold(prediction, lowerThreshold=2, upperThreshold=2, insideValue=1, outsideValue=0)
    edema_gt = sitk.BinaryThreshold(ground_truth, lowerThreshold=2, upperThreshold=2, insideValue=1, outsideValue=0)
    edema_and_tumor_pred = sitk.BinaryThreshold(prediction, lowerThreshold=1, upperThreshold=2, insideValue=1, outsideValue=0)
    edema_and_tumor_gt = sitk.BinaryThreshold(ground_truth, lowerThreshold=1, upperThreshold=2, insideValue=1, outsideValue=0)
    measures1 = sitk.LabelOverlapMeasuresImageFilter()
    measures1.Execute(tumor_pred, tumor_gt)
    measures2 = sitk.LabelOverlapMeasuresImageFilter()
    measures2.Execute(edema_pred, edema_gt)
    measures3 = sitk.LabelOverlapMeasuresImageFilter()
    measures3.Execute(edema_and_tumor_pred, edema_and_tumor_gt)
    out_dict = {}
    for label, measure in [('Tumor(only)', measures1), ('Edema(only)', measures2), ('Edema and Tumor', measures3)]:
        out_dict[label] = {'Dice': measure.GetDiceCoefficient(), 'FalsePositiveError': measure.GetFalsePositiveError(), 
                           'FalseNegativeError': measure.GetFalseNegativeError(), 'Jaccard': measure.GetJaccardCoefficient()}
    return out_dict    

In [ ]:
def get_seeds(sitk_image, mask=None, further_arguments):
    """Extract seed points from image.

    Extracts image pixels which will serve as seed points for region growing.
    
    #####
    Todo: Implement a method to extract seed points from the image.
    #####
    
    
    Parameters
    ----------
    sitk_image : sitk.Image
        input image.
    mask : sitk.Image | None
        The optional mask. May be used to limit the seed points to a certain region.
    further_arguments : If you need further arguments, you can add them here.
        
        
    Returns
    -------
    seeds: List
        List of extracted seed-points in pixel-ccordinates [(x1,y1,z1),(x1,y2,z2),...,(xn,yn,zn)].
    """
    return seeds

In [ ]:
def preprocess_for_edema_segmentation(img):
    """ Pre-process the image before edema segmentation.
    
    Parameters
     ----------
     img : sitk.Image
         input image.
     
     Returns
     -------
     img : sitk.Image
         pre-processed image.
     """
    return processed_img

def segment_edema(images, arguments_for_get_seeds_function, further_arguments):
    """ Segment the edema region in the image.
     
     Parameters
     ----------
     img : sitk.Image
         input image.
     arguments_for_get_seeds_function : Dict 
          For example: {'param1': value1, 'param2': value2, ...}
     
     Returns
     -------
     edema_segmentation : sitk.Image
         binary image with the segmented edema region.
    """
     # EDEMA Segmentation
     # Choose a modality as input for edema segmentation E.g. img = images['flair']
    img = images['flair']

     #
     # Pre-Process flair image before seed extraction
     # E.g. sitk.RescaleIntensity, sitk.Median, sitk.GradientMagnitude, ... 
     #
    preprocessed_img = preprocess_for_edema_segmentation(img)
     
     #
     # Seed extraction from flair image
     #
    seeds = get_seeds(preprocessed_img, **arguments_for_get_seeds_function)
     
     # Get some information about the seeds (optional)
    print(f"Found {len(seeds)} seeds for region-growing.")
     
     #
     # Set up sitk.ConnectedThresholdImageFilter (region growing) for segmentation with found seeds
     # segmentor = sitk.ConnectedThresholdImageFilter()
     # segmentor.SetLower(0)  # lower threshold for region growing 
     # Likewise set other parameters of the region growing algorithm and set the seeds.
     # edema_segmentation = segmentor.Execute(preprocessed_img)
     
     #
     # Post-process found segmentation. Morphological operations, etc.
     # E.g. sitk.BinaryMorphologicalClosing, sitk.BinaryMorphologicalOpening, sitk.BinaryFillhole, ...
     #
    return edema_segmentation
     
     
def preprocess_for_tumor_segmentation(img):
    """ Pre-process the image before tumor segmentation.
    
    Parameters
     ----------
     img : sitk.Image
         input image.
     
     Returns
     -------
     img : sitk.Image
         pre-processed image.
    """
    return processed_img
 
def segment_tumor(img):
    """Segment the tumor region in the image.
     
    """
    return tumor_segmentation

# sitk.WriteImage(edema_segmentation, os.path.join(OUTDIR, "edema.nii.gz"))
print("Done.")
edema_segmentation = segment_edema(images, arguments_for_get_seeds_function, further_arguments)
measures = sitk.LabelOverlapMeasuresImageFilter()
measures.Execute(edema_segmentation, sitk.BinaryThreshold(images['label'], lowerThreshold = 1))
print(measures.GetDiceCoefficient(1), measures.GetFalseNegativeError(1), measures.GetFalsePositiveError(1))

eval_dict = evaluate_labels(edema_segmentation*2, images['label'])
print(eval_dict)
 

In [ ]:
#
# Visualize image overlayed with segmentation/masks
# running this cell may take some time uncomment only if needed
#
#ImageSliceViewer3D(flair_image_smooth, seed_image, figsize=(3,3))
ImageSliceViewer3D(img, edema_segmentation, figsize=(3,3))